In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import TensorBoard,EarlyStopping


import os
current_dir = os.getcwd()
%matplotlib inline

In [ ]:
class DataPreparation():
    def __init__(self):
        self.data = None
        self.x_train = None
        self.y_train = None
        self.x_test = None
        self.y_test = None
        self.num_classes = None
        
        
    def load_data(self):
        (self.x_train, self.y_train), (self.x_test, self.y_test) = tf.keras.datasets.mnist.load_data(path=os.path.join(current_dir,'data','mnist.npz'))

    '''
    In Keras, the layers used for two-dimensional convolutions expect pixel values with the dimensions 
    [pixels][width][height][channels].
    We’ll use the .reshape() method to perform this action. 
    Finally, normalize the image data by dividing each pixel value by 255.
    (since RGB value can range from 0 to 255)
    reshape to be [samples][width][height][channels]
    
    we need to convert the dependent variable in the form of integers to a binary class matrix. 
    This can be achieved by the to_categorical() function

    '''    
    def reshape_data(self):
        img_rows, img_cols = 28, 28
        print("Before: ")
        print(self.x_train.shape)
        print(self.x_test.shape)
        self.x_train = self.x_train.reshape(self.x_train.shape[0],img_rows,img_cols,1).astype('float32')
        self.x_test = self.x_test.reshape(self.x_test.shape[0],img_rows,img_cols,1).astype('float32')
        self.x_train /= 255
        self.x_test /= 255
        print("After: ")
        print(self.x_train.shape)
        print(self.x_test.shape)
        self.y_train = to_categorical(self.y_train)
        self.y_test = to_categorical(self.y_test)
        self.num_classes = self.y_test.shape[1]
        print(self.num_classes)
        

In [ ]:
dp_obj = DataPreparation()     
dp_obj.load_data()
dp_obj.reshape_data()

1. The first hidden layer is a convolutional layer called a Convolution2D. The layer has 32 feature maps, which with the size of 5×5 and a rectifier activation function. This is the input layer, expecting images with the structure outline above [pixels][width][height].
2. Next we define a pooling layer that takes the max called MaxPooling2D. It is configured with a pool size of 2×2.
3. The next layer is a regularization layer using dropout called Dropout. It is configured to randomly exclude 20% of neurons in the layer in order to reduce overfitting.
4. Next is a layer that converts the 2D matrix data to a vector called Flatten. It allows the output to be processed by standard fully connected layers.
5. Next a fully connected layer with 128 neurons and rectifier activation function.
6. Finally, the output layer has 10 neurons for the 10 classes and a softmax activation function to output probability-like predictions for each class.

In [ ]:
class ModelDesign():
    def __init__(self):
        self.model = None
        self.num_classes = dp_obj.num_classes
        self.history = None
        self.keras_callbacks_list = []
        '''
        if apply_tensorboard_callbacks:
            tensorboard = TensorBoard(log_dir='./logs',histogram_freq=1,write_images=True)
            self.keras_callbacks = [tensorboard]
        '''
    def set_call_back(self,apply_callback,tensorboard_callback):
        if apply_callback:
            es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=60)
            self.keras_callbacks_list.append(es)
        if tensorboard_callback:
            tensorboard = TensorBoard(log_dir='./logs',histogram_freq=1,write_images=True)
            self.keras_callbacks_list.append(tensorboard)
        
    def base_model(self):
        self.model = Sequential()
        self.model.add(Conv2D(32,(5,5),activation='relu'))
        self.model.add(MaxPooling2D(pool_size=(2,2)))
        self.model.add(Dropout(0.2))
        self.model.add(Flatten())
        self.model.add(Dense(self.num_classes,activation="softmax"))
        
    def compile_model(self):
        self.model.compile(loss="categorical_crossentropy",optimizer="adam",metrics=["accuracy"])
    
    def train_model(self,batch_size,epochs):
        x_train = dp_obj.x_train
        y_train = dp_obj.y_train
        x_test = dp_obj.x_test
        y_test = dp_obj.y_test        
        self.history = self.model.fit(x_train,y_train, batch_size=batch_size, epochs=epochs, 
                                      verbose=2, validation_data = (x_test,y_test),shuffle=True,
                                     callbacks=self.keras_callbacks_list)

In [ ]:
epochs = 100
batch_size = 128

md_obj = ModelDesign()
"You can skip the line or set the flag according to your needs"
md_obj.set_call_back(True,True)
md_obj.base_model()
md_obj.compile_model()
md_obj.train_model(batch_size,epochs)

In [ ]:
score = md_obj.model.evaluate(dp_obj.x_test, dp_obj.y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
class EvaluateModel():
    def __init__(self,history):
        self.history = history
        self.epochs = range(len(epochs))
        
    def plot_acc_loss(self):
        
        acc = history.history['acc']
        loss = history.history['loss']

        plt.plot(epochs, acc, 'b', label='Training accuracy')
        plt.title('Training accuracy')

        plt.figure()

        plt.plot(epochs, loss, 'b', label='Training Loss')
        plt.title('Training loss')
        plt.legend()

        plt.show()

em_obj = EvaluateModel(md_obj.history)
em_obj.plot_acc_loss()

In [ ]:
from PIL import Image
from IPython.display import Image as IMG


class Prediction():
    
    def __init__(self,model):
        self.model = model
    
    def predict(self,image):
        img = Image.open(image)
        print("Image: ")
        display(IMG(filename=image))
        img = img.convert("L")
        img = img.resize((28,28))
        im2arr = np.array(img)
        im2arr = im2arr.reshape(1,28,28,1)
        y_pred = self.model.predict(im2arr)
        return y_pred
        
pred_obj = Prediction(md_obj.model)
files = None

for r,d,f in os.walk(os.path.join(current_dir,"Test_Data")):
    files = f
for item in files:
    result = pred_obj.predict(os.path.join(current_dir,"Test_Data",item))
    print("Predition Label: ",result.argmax(),'\n')
    #print(type(result))